# Convert dataset to tf record
As described in https://stackoverflow.com/questions/46820500/how-to-handle-large-amouts-of-data-in-tensorflow/47040165#47040165

In [ ]:
import numpy as np
import tensorflow as tf
import os
import sys
from pathlib import Path

## Data folder

In [ ]:
# dataset_name = "lastfm_10_pc"
dataset_name = "ml-10m"

In [ ]:
# Adjust your path accordingly
data_folder = Path("/Users/nknyazev/Documents/Delft/Thesis/temporal/data/processed/final/").joinpath(dataset_name)

In [ ]:
subfolders = ["train", "validation", "test"]

In [ ]:
subfolder_paths = [data_folder.joinpath(x) for x in subfolders]

In [ ]:
arrays = ["X", "y", "user_ids", "seq_lens"]

In [ ]:
assert os.path.exists(data_folder)

## Conversion to TFRecords

In [ ]:
def subset_to_tfrecords(output_file, **kwargs):
    num_samples = len(kwargs["y"])
    
    print("Opening writer.")
    writer = tf.python_io.TFRecordWriter(output_file)
    
    for i in range(num_samples):
        if i % 1000 == 0:
            print("Completed {}".format(i))
        entry = {k:v[i] for k,v in kwargs.items()}
        serialized = entry_to_tfrecords(**entry)
        writer.write(serialized)
    print("Closing writer.")
    writer.close()

In [ ]:
def entry_to_tfrecords(X, y, seq_lens, user_ids):
    feature = {
        'X': tf.train.Feature(int64_list=tf.train.Int64List(value=X.flatten())),
        'y': tf.train.Feature(int64_list=tf.train.Int64List(value=y)),
        'seq_lens': tf.train.Feature(int64_list=tf.train.Int64List(value=[seq_lens])),
        'user_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=[user_ids]))

    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    serialized = example.SerializeToString()
    return serialized
    

In [ ]:
for subfolder_path in subfolder_paths:
    subfolder_arrays = {k:np.load(str(subfolder_path.joinpath(k)) + ".npy") for k in arrays}
    subset_to_tfrecords(**subfolder_arrays, output_file=str(subfolder_path.joinpath("{}.tfrecords".format(subfolder_path.stem))))
    